**LOAD DATASET**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv")
data.head()

**FANTASY TEAM BASED ON ALL MATCHES FROM 2008-2020**

In [ ]:
batsman = data[["id","batsman", "batsman_runs","non_boundary", "is_wicket"]]

In [ ]:
matches = batsman.groupby(["id","batsman"]).size().reset_index()
matches = matches[["id","batsman"]]
number_of_matches = pd.DataFrame(matches["batsman"].value_counts())
number_of_matches.columns=['matches']

In [ ]:
total_runs = pd.DataFrame(batsman.groupby('batsman')['batsman_runs'].sum())
total_runs.columns=['runs']

In [ ]:
u_batsman = pd.merge(number_of_matches, total_runs, left_index=True, right_index=True)
u_batsman["runperatch"] = u_batsman["runs"]/u_batsman["matches"] 

In [ ]:
fielder = data["fielder"]
fielder.dropna()
catches = pd.DataFrame(fielder.value_counts())

In [ ]:
player = pd.merge(u_batsman, catches, left_index=True, right_index=True)
player.columns = ['n_match','t_runs','per_match_runs','catches']
player["catchespermatch"] = player["catches"]/player["n_match"]
player.columns = ['n_match','t_runs','per_match_runs','catches','per_match_catches']

In [ ]:
bowler = data[["id","bowler", "is_wicket"]]

In [ ]:
matches = bowler.groupby(["id","bowler"]).size().reset_index()
matches = matches[["id","bowler"]]
number_of_matches = pd.DataFrame(matches["bowler"].value_counts())
number_of_matches.columns=['matches']

In [ ]:
total_wickets = pd.DataFrame(bowler.groupby('bowler')['is_wicket'].sum())
total_wickets.columns=['wickets']

In [ ]:
u_bowler = pd.merge(number_of_matches, total_wickets, left_index=True, right_index=True)
u_bowler["wicketspermatch"] = u_bowler["wickets"]/u_batsman["matches"] 
u_bowler = u_bowler[["wickets", "wicketspermatch"]]

In [ ]:
#scoring - (score per run =1), (score per catch = 8), (score per wicket = 25)
player = pd.merge(player, u_bowler, left_index=True, right_index=True)
batsman = player.index
player["score"]= player["t_runs"] + player["catches"]*8 + player["wickets"]*25
player = player[player['n_match']>=3]

In [ ]:
player = player.sort_values(by=['score','n_match'], ascending= False)

In [ ]:
print("Fantasy Team of top 11 players - IPL 2008-2020")
player.head(11)

**FANTASY TEAM BASED ON MATCHES OF 2020**

1. Extract Data only of the year 2020

In [ ]:
data = pd.read_csv("/kaggle/input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv")
u_data = pd.DataFrame()
for date in data['date']:
    if date.split('-')[0] == '2020':
        u_data = u_data.append(data[data['date']==date])
match_id = u_data['id']
match_id = match_id.reset_index()
match_id = match_id['id']
match_id = list(match_id)

In [ ]:
data = pd.read_csv("/kaggle/input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv")
u_data = pd.DataFrame()
# data[data['id'] in match_id]
l = len(data)
for id in match_id:
    u_data = u_data.append(data[data['id'] == id])
data = u_data

2. Process data to create a fantasy team

In [ ]:
batsman = data[["id","batsman", "batsman_runs","non_boundary", "is_wicket"]]

In [ ]:
matches = batsman.groupby(["id","batsman"]).size().reset_index()
matches = matches[["id","batsman"]]
number_of_matches = pd.DataFrame(matches["batsman"].value_counts())
number_of_matches.columns=['matches']

In [ ]:
total_runs = pd.DataFrame(batsman.groupby('batsman')['batsman_runs'].sum())
total_runs.columns=['runs']

In [ ]:
u_batsman = pd.merge(number_of_matches, total_runs, left_index=True, right_index=True)
u_batsman["runperatch"] = u_batsman["runs"]/u_batsman["matches"] 

In [ ]:
fielder = data["fielder"]
fielder.dropna()
catches = pd.DataFrame(fielder.value_counts())

In [ ]:
player = pd.merge(u_batsman, catches, left_index=True, right_index=True)
player.columns = ['n_match','t_runs','per_match_runs','catches']
player["catchespermatch"] = player["catches"]/player["n_match"]
player.columns = ['n_match','t_runs','per_match_runs','catches','per_match_catches']

In [ ]:
bowler = data[["id","bowler", "is_wicket"]]

In [ ]:
matches = bowler.groupby(["id","bowler"]).size().reset_index()
matches = matches[["id","bowler"]]
number_of_matches = pd.DataFrame(matches["bowler"].value_counts())
number_of_matches.columns=['matches']

In [ ]:
total_wickets = pd.DataFrame(bowler.groupby('bowler')['is_wicket'].sum())
total_wickets.columns=['wickets']

In [ ]:
total_wickets = pd.DataFrame(bowler.groupby('bowler')['is_wicket'].sum())
total_wickets.columns=['wickets']

In [ ]:
u_bowler = pd.merge(number_of_matches, total_wickets, left_index=True, right_index=True)
u_bowler["wicketspermatch"] = u_bowler["wickets"]/u_batsman["matches"] 
u_bowler = u_bowler[["wickets", "wicketspermatch"]]

In [ ]:
#scoring - (score per run =1), (score per catch = 8), (score per wicket = 25)
player = pd.merge(player, u_bowler, left_index=True, right_index=True)
batsman = player.index
player["score"]= player["t_runs"] + player["catches"]*8 + player["wickets"]*25
player = player[player['n_match']>=3]

In [ ]:
player = player.sort_values(by=['score','n_match'], ascending= False)

In [ ]:
print("Fantasy Team of top 11 players - IPL 2020")
player.head(11)